In [1]:
import math
import ipywidgets
import time

prc_time_widget = ipywidgets.FloatText(description="소요시간(초)")
opt_prc_widget = ipywidgets.FloatText(description="옵션가")

def get_opt_prc_one_thd(opt_type, strike_prc, spot_prc, remain_day, interest_rate, volatility, node_cnt):
    st_time = time.time()
    
    remain_year = remain_day / 365
    delta_t = max(remain_year / node_cnt, 0.000001)
    a = math.exp(interest_rate * 0.01 * delta_t)
    u = math.exp(volatility * 0.01 * math.sqrt(delta_t))
    d = 1 / u
    p = (a - d) / (u - d)
    r_p = 1 - p

    datas = [0] * (node_cnt + 1)

    for row in range(node_cnt+1):
        datas[row] = spot_prc * (u**(node_cnt-row)) * (d**row)
        datas[row] = max(datas[row]-strike_prc, 0) if "call" == opt_type else max(strike_prc-datas[row], 0)
    
    print(datas)

    for col in range(node_cnt, 0, -1):
        for row in range(col):
            datas[row] = (p * datas[row] + r_p * datas[row+1]) / a
    
    ed_time = time.time()
    
    opt_prc_widget.value = datas[0]
    prc_time_widget.value = ed_time - st_time

ipywidgets.interact_manual(
    get_opt_prc_one_thd,
    opt_type=ipywidgets.Dropdown(options=[("콜", "call"), ("풋", "put")], description="옵션타입"),
    spot_prc=ipywidgets.BoundedFloatText(min=0, max=1000, value=274.34, step=0.01, description="기초자산"),
    strike_prc=ipywidgets.BoundedFloatText(min=0, max=1000, value=275, step=2.5, description="행사가"),
    remain_day=ipywidgets.BoundedIntText(min=1, max=1000, value=31, description="잔존일"),
    interest_rate=ipywidgets.BoundedFloatText(min=0, value=1.55, step=0.01, description="이자율(%)"),
    volatility=ipywidgets.BoundedFloatText(min=0.01, value=13.59, step=0.01, description="변동성(%)"),
    node_cnt=ipywidgets.BoundedIntText(min=2, max=100000, value=49, description="노드수"),
);

display(prc_time_widget)
display(opt_prc_widget)

interactive(children=(Dropdown(description='옵션타입', options=(('콜', 'call'), ('풋', 'put')), value='call'), Bound…

FloatText(value=0.0, description='소요시간(초)')

FloatText(value=0.0, description='옵션가')

In [2]:
import math
import ipywidgets
import time
import numpy as np
import pycuda.autoinit
import pycuda.driver as drv
from pycuda import compiler

OPT_TYPE_CALL = 0
OPT_TYPE_PUT = 1

# Kernel code
kernel_code = """
__constant__ int opt_type;
__constant__ float params[7];

__global__ void get_opt_prc_n_thd(float* ret)
{
    extern __shared__ float datas[];
    
    int node_cnt = blockDim.x - 1;
    int tid = threadIdx.x;
    
    const float spot_prc = params[0];
    const float strike_prc = params[1];
    const float a = params[2];
    const float u = params[3];
    const float d = params[4];
    const float p = params[5];
    const float r_p = params[6];
    
    datas[tid] = spot_prc * powf(u, node_cnt-tid) * powf(d, tid);
    datas[tid] = ( 0 == opt_type ) ? fmaxf(datas[tid]-strike_prc, 0) : fmaxf(strike_prc-datas[tid], 0);
    
    __syncthreads();
    
    int cnt = node_cnt;
    
    
    
    ret[tid] = datas[tid];
}
"""

# Compile the kernel code
mod = compiler.SourceModule(kernel_code)

# result
#result = np.zeros(1, dtype=np.float)

opt_type = OPT_TYPE_CALL
spot_prc = 274.34
strike_prc = 275.0
remain_day = 31
interest_rate = 1.55
volatility = 13.59
node_cnt = 4

remain_year = remain_day / 365
delta_t = max(remain_year / node_cnt, 0.000001)
a = math.exp(interest_rate * 0.01 * delta_t)
u = math.exp(volatility * 0.01 * math.sqrt(delta_t))
d = 1 / u
p = (a - d) / (u - d)
r_p = 1 - p

print(u)
print(d)

params = [spot_prc, strike_prc, a, u, d, p, r_p]

inputs_int = np.array([opt_type], dtype=np.int)
inputs_float = np.array(params, dtype=np.float32)

drv.memcpy_htod(mod.get_global("opt_type")[0], inputs_int)
drv.memcpy_htod(mod.get_global("params")[0], inputs_float)

# Get kernel function
get_opt_func = mod.get_function("get_opt_prc_n_thd")

# Run & Print
thd_cnt = node_cnt + 1
result_arr = np.zeros(thd_cnt, dtype=np.float32)
get_opt_func(drv.InOut(result_arr), block=(thd_cnt, 1, 1), shared=thd_cnt)

print(inputs_float)
print(result_arr)

1.0200000493944084
0.9803921093863841
[274.34       275.           1.0003291    1.0200001    0.9803921
   0.50336003   0.49663997]
[21.95456  10.423401  0.        0.        0.      ]


c:\users\junhyeong\anaconda3\envs\py36_64\lib\site-packages\ipykernel_launcher.py:42: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc warning : nvcc support for Microsoft Visual Studio 2010 and earlier has been deprecated and is no longer being maintained
kernel.cu
c:\program files\nvidia gpu computing toolkit\cuda\v8.0\include\math_functions.h : warning C4819: ���� �ڵ� ������(949)���� ǥ���� �� ���� ���ڰ� ���Ͽ� ��� �ֽ��ϴ�. �����Ͱ� �սǵ��� �ʰ� �Ϸ��� �ش� ������ �����ڵ� �������� �����Ͻʽÿ�.
c:\program files\nvidia gpu computing toolkit\cuda\v8.0\include\math_functions.h(849) : warning C4819: ���� �ڵ� ������(949)���� ǥ���� �� ���� ���ڰ� ���Ͽ� ��� �ֽ��ϴ�. �����Ͱ� �սǵ��� �ʰ� �Ϸ��� �ش� ������ �����ڵ� �������� �����Ͻʽÿ�.
c:\program files\nvidia gpu computing toolkit\cuda\v8.0\include\math_functions